# **Pacotes e configurações antes da execução.**

In [ ]:
# importar o arquivo diretamente de seu google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
#A primeira linha instala o Java 8 na maquina da sessão
#A segunda faz o download do spark no link escolhido
#A terceira extrai o arquivo baixado

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.4.2/spark-3.4.2-bin-hadoop3.tgz
!tar xf spark-3.4.2-bin-hadoop3.tgz

In [ ]:
#E a quarta instala a biblioteca findspark para facilitar na criação da sessão spark

!pip install -q findspark

In [ ]:
#Agora iremos utilizar o código abaixo para configurar as variáveis de caminho de ambiente utilizando a biblioteca "os"

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.2-bin-hadoop3"

In [ ]:
#criando uma copia do dataframe na minha pasta drive
!cp /content/drive/MyDrive/Colab_Notebooks/imdb-reviews-pt-br.csv /content

In [ ]:
#importando e iniciando o findspark e importando o sparksession
import findspark

findspark.init()

from pyspark.sql import SparkSession

In [ ]:
#criando a sessao
spark = SparkSession.builder.master("local[*]").getOrCreate()

# **Prática 01 – Somatório de IDs.**
**Qual o valor da soma de todos os campos “id” dos filmes classificados como negativos para o banco de dados “imdb-reviews-pt-br.csv”**


In [ ]:
#carregando o conjunto de dados
df = spark.read.csv("imdb-reviews-pt-br.csv", header=True, quote="\"", escape="\"", encoding="UTF-8")

In [ ]:
#mostrando as 20 primeiras linhas
df.show()

+---+--------------------+--------------------+---------+
| id|             text_en|             text_pt|sentiment|
+---+--------------------+--------------------+---------+
|  1|Once again Mr. Co...|Mais uma vez, o S...|      neg|
|  2|This is an exampl...|Este é um exemplo...|      neg|
|  3|First of all I ha...|Primeiro de tudo ...|      neg|
|  4|Not even the Beat...|Nem mesmo os Beat...|      neg|
|  5|Brass pictures mo...|Filmes de fotos d...|      neg|
|  6|A funny thing hap...|Uma coisa engraça...|      neg|
|  7|This German horro...|Este filme de ter...|      neg|
|  8|Being a long-time...|Sendo um fã de lo...|      neg|
|  9|"Tokyo Eyes" tell...|"Tokyo Eyes" fala...|      neg|
| 10|Wealthy horse ran...|Fazendeiros ricos...|      neg|
| 11|Cage plays a drun...|Cage interpreta u...|      neg|
| 12|First of all, I w...|Primeiro de tudo,...|      neg|
| 13|So tell me - what...|Então me diga - q...|      neg|
| 14|A big disappointm...|Uma grande decepç...|      neg|
| 15|This film

In [ ]:
#print do schema
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- text_en: string (nullable = true)
 |-- text_pt: string (nullable = true)
 |-- sentiment: string (nullable = true)



In [ ]:
#confirmando o tipo do dataframe
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
#contando  as linhas do conjunto de dados
df.count()

17302

In [ ]:
# importando os métodos com funções para transformações de variáveis
from pyspark.sql.functions import *

In [ ]:
#Transformando a coluna id em integer.
df = df.withColumn(
    colName = 'id',
    col = regexp_replace('id','','').cast('int')
    )

# printando o resultado
df.select('id').printSchema()

In [ ]:
#Criando variavel apenas com coluna id e sentiment
df_Select = df.select('id','sentiment')

In [ ]:
#Mostrando o resultado
df_Select.show()

In [ ]:
#Filtrando por sentiment = neg
df_Neg = df_Select.where('sentiment == "neg" ')


In [ ]:
#Mostrando resultado
print("Palavras filtradas com sentimento = neg")
df_Neg.show()

In [ ]:
#somando a coluna id com base no sentiment neg.
total_neg = df_Neg.groupBy('sentiment').agg(sum('id').alias("Total_Id_Neg")).orderBy('sentiment').where('sentiment == "neg" ')
total_neg.show()

In [ ]:
#Pegando o valor da coluna Total_Id_Neg e printando resultado
soma_neg = total_neg.select(col("Total_Id_Neg")).first()[0]
print(f'Valor do somatorio dos ids negativos: {soma_neg}')

# **O valor da soma do id com base no sentiment = neg é: 459568555**

---

# **Prática 02 – Diferença do número de palavras totais de português para inglês dos textos negativos**

**Contar palavras dos textos negativos e achar diferença de quantidade.**

In [ ]:
#selecionando a coluna desejada
df_en = df.select('id','text_en','sentiment')
df_pt = df.select('id','text_pt','sentiment')

In [ ]:
#filtrando o conjunto de dados para o sentiment = neg
df_en = df_en.where('sentiment == "neg" ')
df_pt = df_pt.where('sentiment == "neg" ')

In [ ]:
#removendo a coluna sentiment
df_en = df_en.drop('sentiment')
df_pt = df_pt.drop('sentiment')

In [ ]:
df_en.show()
df_pt.show()

In [ ]:
# Usa as funções explode e split para dividir o texto em palavras
palavras_pt = df_pt.select("id", explode(split("text_pt", "[ ]+")).alias("palavra"))
palavras_en = df_en.select("id", explode(split("text_en", "[ ]+")).alias("palavra"))

# Usa a função count para contar as ocorrências de cada palavra ordenando do maior para o menor
contagem_pt = palavras_pt.groupBy("palavra").count().orderBy(desc("count"))
contagem_en = palavras_en.groupBy("palavra").count().orderBy(desc("count"))

# Mostra o resultado
print("Palavras em Portugues")
contagem_pt.show()
print("Palavras em Ingles")
contagem_en.show()

In [ ]:
#Soma dos valores
contagem_pt.agg(sum('count').alias("Total_palavras_pt")).show()
contagem_en.agg(sum('count').alias("Total_palavras_en")).show()

In [ ]:
#pegando os valores para fazer a subtração
tot_pt = contagem_pt.agg(sum('count').alias("Total_palavras_pt")).first()[0]
tot_en = contagem_en.agg(sum('count').alias("Total_palavras_en")).first()[0]

In [ ]:
tot_pt, tot_en

In [ ]:
print(f'O texto em Portugues possui {tot_pt - tot_en} palavras a mais que o ingles')

# **O texto em Portugues possui 54997 palavras a mais que o ingles**